In [2]:
import logging
import os
import pickle
import numpy as np

from sklearn.preprocessing import RobustScaler

from data_ops.preprocessing import permute_by_pt
from data_ops.preprocessing import extract
from data_ops.preprocessing import sequentialize_by_pt
from data_ops.preprocessing import randomize
from data_ops.preprocessing import rewrite_content

In [ ]:
data_dir = '../../'
#filename = 'antikt-kt-train.pickle'
filename = 'antikt-kt-pileup25-new-train.pickle'
path_to_preprocessed_dir = os.path.join(data_dir, 'preprocessed')
path_to_preprocessed = os.path.join(path_to_preprocessed_dir, filename)

In [ ]:
with open(os.path.join(data_dir, 'raw', filename), mode="rb") as fd:
            X, y = pickle.load(fd, encoding='latin-1')

In [ ]:
tf = load_tf(args.data_dir, "{}-train.pickle".format(args.filename))
    X, y = load_data(args.data_dir, "{}-train.pickle".format(args.filename))
    for ij, jet in enumerate(X):
        jet["content"] = tf.transform(jet["content"])

    if args.n_train > 0:
        indices = torch.randperm(len(X)).numpy()[:args.n_train]
        X = [X[i] for i in indices]
        y = y[indices]

In [ ]:
len(X)

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
m = nn.Threshold(0.5, 0)
input = Variable(torch.randn(2))
print(input)
print(m(input))

In [ ]:
m = nn.Softmax()
input = Variable(torch.randn(2, 3))
print(input)
print(m(input))

In [ ]:
(input > 0.5).type(torch.FloatTensor)

In [ ]:
input

In [ ]:
A = torch.FloatTensor(input.size()).zero_()+1
B = torch.FloatTensor(input.size()).zero_()

In [ ]:
def loss_adversarial(y_pred, y):
        return -(y * torch.log(y_pred) + (1. - y) * torch.log(1. - y_pred))

In [ ]:
loss_adversarial(A,B)

In [ ]:
torch.log(A)

In [3]:
import torch
from torch.optim import Adam#, lr_scheduler
import copy
import numpy as np
import logging
import pickle
import time
import os
import argparse
from torch.autograd import Variable

from sklearn.model_selection import train_test_split

from data_ops.wrapping import wrap
from data_ops.wrapping import unwrap
from data_ops.wrapping import wrap_X
from data_ops.wrapping import unwrap_X

from misc.constants import *
from misc.handlers import ExperimentHandler
from misc.loggers import StatsLogger

from monitors.losses import *
from monitors.monitors import *

from architectures import PredictFromParticleEmbedding
#from architectures import AdversarialParticleEmbedding

from loading import load_data
from loading import load_tf
from loading import crop

In [ ]:
filename = 'antikt-kt'
data_dir = '../../'
tf = load_tf(data_dir, "{}-train.pickle".format(filename))
X, y = load_data(data_dir, "{}-train.pickle".format(filename))
for ij, jet in enumerate(X):
    jet["content"] = tf.transform(jet["content"])

In [4]:
''' ARGUMENTS '''
'''----------------------------------------------------------------------- '''
parser = argparse.ArgumentParser(description='Jets')

# data args
parser.add_argument("-f", "--filename", type=str, default='antikt-kt')
parser.add_argument("--data_dir", type=str, default=DATA_DIR)
parser.add_argument("-n", "--n_train", type=int, default=-1)
parser.add_argument("--n_valid", type=int, default=27000)
parser.add_argument("--dont_add_cropped", action='store_true', default=False)
parser.add_argument("-p", "--pileup", action='store_true', default=False)

# general model args
parser.add_argument("-m", "--model_type", help="index of the model you want to train - look in constants.py for the model list", type=int, default=1)
parser.add_argument("--features", type=int, default=7)
parser.add_argument("--hidden", type=int, default=40)

# logging args
parser.add_argument("-s", "--silent", action='store_true', default=False)
parser.add_argument("-v", "--verbose", action='store_true', default=False)
parser.add_argument("--extra_tag", type=int, default=0)

# loading previous models args
parser.add_argument("-l", "--load", help="model directory from which we load a state_dict", type=str, default=None)
parser.add_argument("-r", "--restart", help="restart a loaded model from where it left off", action='store_true', default=False)

# training args
parser.add_argument("-e", "--epochs", type=int, default=50)
parser.add_argument("-b", "--batch_size", type=int, default=100)
parser.add_argument("-a", "--step_size", type=float, default=0.001)
parser.add_argument("-d", "--decay", type=float, default=.94)

# computing args
parser.add_argument("--seed", help="Random seed used in torch and numpy", type=int, default=None)
parser.add_argument("-g", "--gpu", type=str, default="")

# MPNN
parser.add_argument("--not_leaves", action='store_true')
parser.add_argument("-i", "--iters", type=int, default=0)

# email
parser.add_argument("--sender", type=str, default="results74207281@gmail.com")
parser.add_argument("--password", type=str, default="deeplearning")

# debugging
parser.add_argument("--debug", help="sets everything small for fast model debugging. use in combination with ipdb", action='store_true', default=False)

args = parser.parse_args()

In [ ]:
if args.debug:
    args.hidden = 1
    args.batch_size = 9
    args.verbose = True
    args.epochs = 3
    args.n_train = 1000
    args.seed = 1
    args.iters = 1

In [5]:
_, Transform, model_type = TRANSFORMS[args.model_type]

In [6]:
model_type

'recnn/simple'

In [7]:
args.root_exp_dir = os.path.join(MODELS_DIR,model_type, str(args.iters))
args.recipient = 'psn240@nyu.edu'
eh = ExperimentHandler(args)

12/10/2017 08:59:34 AM 	batch_size = 100
12/10/2017 08:59:34 AM 	data_dir = data/w-vs-qcd/pickles
12/10/2017 08:59:34 AM 	debug = False
12/10/2017 08:59:34 AM 	decay = 0.94
12/10/2017 08:59:34 AM 	dont_add_cropped = False
12/10/2017 08:59:34 AM 	epochs = 50
12/10/2017 08:59:34 AM 	extra_tag = 0
12/10/2017 08:59:34 AM 	features = 7
12/10/2017 08:59:34 AM 	filename = /Users/Bangz/Library/Jupyter/runtime/kernel-19e504c5-4cef-417d-bfbc-287b45703803.json
12/10/2017 08:59:34 AM 	gpu = 
12/10/2017 08:59:34 AM 	hidden = 40
12/10/2017 08:59:34 AM 	iters = 0
12/10/2017 08:59:34 AM 	load = None
12/10/2017 08:59:34 AM 	model_type = 1
12/10/2017 08:59:34 AM 	n_train = -1
12/10/2017 08:59:34 AM 	n_valid = 27000
12/10/2017 08:59:34 AM 	not_leaves = False
12/10/2017 08:59:34 AM 	password = deeplearning
12/10/2017 08:59:34 AM 	pileup = False
12/10/2017 08:59:34 AM 	recipient = psn240@nyu.edu
12/10/2017 08:59:34 AM 	restart = False
12/10/2017 08:59:34 AM 	root_exp_dir = models/recnn/simple/0
12/10/2017 

In [26]:
logdict = dict(
                epoch=1,
                iteration=1,
                yy=[1.0],
                yy_pred=[1,0],
                w_valid=[1, 2, 3],
                #w_valid=w_valid,
                train_loss=0.5,
                valid_loss=0.5,
                settings=settings,
                #model=model
            )

In [27]:
model_kwargs = {
            'features': args.features,
            'hidden': args.hidden,
            'iters': args.iters,
            'leaves': 10,
            'batch' : args.batch_size,
        }
        
settings = {
            "transform": Transform,
            #"predict": Predict,
            "model_kwargs": model_kwargs,
            "step_size": args.step_size,
            "args": args,
            }

In [33]:
roc_auc = ROCAUC()
inv_fpr = InvFPR()
best_inv_fpr = Best(inv_fpr)
epoch_counter = Regurgitate('epoch')
batch_counter = Regurgitate('iteration')
valid_loss = Regurgitate('valid_loss')
train_loss = Regurgitate('train_loss')

In [37]:
monitors = [
            epoch_counter,
            batch_counter,
            roc_auc,
            inv_fpr,
            best_inv_fpr,
            valid_loss,
            train_loss,
        ]

In [38]:
monitors = {m.name: m for m in monitors}

In [39]:
monitors

{'best_inv_fpr': <monitors.monitors.Best at 0x111f45240>,
 'epoch': <monitors.monitors.Regurgitate at 0x111f451d0>,
 'inv_fpr': <monitors.monitors.InvFPR at 0x111f45198>,
 'iteration': <monitors.monitors.Regurgitate at 0x111f450b8>,
 'roc_auc': <monitors.monitors.ROCAUC at 0x111f453c8>,
 'train_loss': <monitors.monitors.Regurgitate at 0x111f45160>,
 'valid_loss': <monitors.monitors.Regurgitate at 0x111f45128>}

In [41]:
stats_dict = {}
for name, monitor in monitors.items():
    stats_dict[name] = monitor(**kwargs)

NameError: name 'kwargs' is not defined

In [49]:
import numpy as np
import matplotlib.pyplot as plt

In [74]:
Y_pred_40 = np.load('../../Y_pred_40.csv.npy')
Y_test_40 = np.load('../../Y_test_40.csv.npy')
Y_pred_50 = np.load('../../Y_pred_50.csv.npy')
Y_test_50 = np.load('../../Y_test_50.csv.npy')
Y_pred_60 = np.load('../../Y_pred_60.csv.npy')
Y_test_60 = np.load('../../Y_test_60.csv.npy')

In [63]:
Y_pred_40 = np.load('../../Y_non_pileup_pred_40.csv.npy')
Y_test_40 = np.load('../../Y_non_pileup_test_40.csv.npy')
Y_pred_50 = np.load('../../Y_non_pileup_pred_50.csv.npy')
Y_test_50 = np.load('../../Y_non_pileup_test_50.csv.npy')
Y_pred_60 = np.load('../../Y_non_pileup_pred_60.csv.npy')
Y_test_60 = np.load('../../Y_non_pileup_test_60.csv.npy')

In [77]:
plt.clf()
plt.hist(Y_pred_40[Y_test_40==0], bins=100, normed=1, histtype="step", label="$p(f(X)=0|Z=40)$", color='red')
plt.hist(Y_pred_50[Y_test_50==0], bins=100, normed=1, histtype="step", label="$p(f(X)=0|Z=50)$", color='blue')
plt.hist(Y_pred_60[Y_test_60==0], bins=100, normed=1, histtype="step", label="$p(f(X)=0|Z=60)$", color='black')
#plt.title('Conditional probabilites densities of the desicision scores at Z = 40, 50, 60')
plt.legend(loc="best")
#plt.ylim(0,4)
plt.xlabel("$f(X)$")
plt.ylabel("$p(f(X))$")
#plt.legend(loc="upper left")
#figure_filename = 
#plot_save(figure_filename)
plt.savefig('Pileup_0.png')
#plt.savefig('Pileup_0.png')
#plt.show() 

In [78]:
plt.clf()
plt.hist(Y_pred_40[Y_test_40==1], bins=100, normed=1, histtype="step", label="$p(f(X)=1|Z=40)$", color='red')
plt.hist(Y_pred_50[Y_test_50==1], bins=100, normed=1, histtype="step", label="$p(f(X)=1|Z=50)$", color='blue')
plt.hist(Y_pred_60[Y_test_60==1], bins=100, normed=1, histtype="step", label="$p(f(X)=1|Z=60)$", color='black')
#plt.title('Conditional probabilites densities of the desicision scores at Z = 40, 50, 60')
#plt.legend(loc="best")
#plt.ylim(0,4)
plt.xlabel("$f(X)$")
plt.ylabel("$p(f(X))$")
plt.legend(loc="upper left")
#figure_filename = 
#plot_save(figure_filename)
plt.savefig('Pileup_1.png')
#plt.savefig('Non_Adv_Pileup_1.png')
#plt.show() 

In [57]:
loss_adv = np.load('../../adv_loss.csv.npy')
loss_rnn = np.load('../../rnn_loss.csv.npy')

In [61]:
len(loss_adv)
len(loss_rnn)

53900

In [62]:
plt.clf()
plt.plot(loss_adv, label="$Advserial loss$", color='red')
plt.plot(loss_rnn, label="$Classifier loss$", color='blue')
plt.legend(loc="best")
plt.savefig('loss.png')